In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
warnings.filterwarnings('ignore')
import regex as re
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '137'
formatted_attribute = 'Weight'
customer_name='%kimballmidwest%'
buckets = """Office Chairs"""

# strategy_version_input=767
# attribute_id_input=2674

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [9]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['image_url']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

168


In [10]:
df=df[(df['external_id'].astype(str)!='69790603')&(df['external_id'].astype(str)!='69790681')&(df['external_id'].astype(str)!='200045714')]
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [11]:
regex_pattern = r"(?i)((?<!support.{0,10})(?<!Minimum.{0,10}?)(?<!Maxi?m?u?m?\s?c?h?i?l?d?.{0,10}?)(?<!accommodates.{10})(?<!light)weigh(?!t\s?capacity)(?!ing).{0,11}(?<!lb.{0,10})lb)|()"               
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [12]:
regex_lbs=r"(?i)(\d+\.?\d*\s?lb)|()"
df['newest_matches']=df['matches'].apply(lambda x: re_extract(regex_lbs, str(x)))
df['newest_matches']=df['newest_matches'].apply(lambda x: natsorted(x))

df_newest=df[df['newest_matches'].astype(str)!='[]']
df_newest['newest_matches']=df_newest['newest_matches'].apply(lambda x: natsorted(x)).apply(lambda x: re.sub(r'','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
curate(df_newest,'newest_matches',formatted_attribute,'ac-km-z')
print(len(df_newest))

Repeated External IDs: []
0


In [13]:
df_newest[curation_col]=df_newest['newest_matches']
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'\[','["',str(x))).apply(lambda x: re.sub(r'\]','"]',str(x))).apply(lambda x: re.sub(r"'",'',str(x))).apply(lambda x: re.sub(r",",'","',str(x))).apply(lambda x: re.sub(r'" ','"',str(x))).apply(lambda x: re.sub(r'(?<=\d)lb',' lb',str(x)))
# rounds(df_newest,curation_col)
# rounding(df_newest,curation_col,'ac-km-z') 

In [14]:
matchesnewest=df_newest[['external_id',curation_col]]
matchesnewest[curation_col]=matchesnewest[curation_col].apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'(?i)(?<!l)b','lb',str(x))).apply(lambda x: re.sub(r'(?<=\d)lb',' lb',str(x)))

# Na

In [18]:
regex_pattern_na=r'(?i)(up.?to.?\d+\s?lb)|(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}weighi?n?g?\s?\d+\s?lb)|(support.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static load capacity)|(\d+\s?lb.?.?\s?capacity)|(store.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static\s?weight\s?capacity)|(users\s?up\s?to\s\d+\s?lb)|(\d+\s?lb.?.?\s?max\s?capacity)|(\d+\s?lb.?.?\s?seating\s?capacity)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(supports\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(\d+\s?lb.?.?\s?Load\s?Capacity)|(weight capacity.?o?f?.?.?\s?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs]\s\d+.?\s?lb.?\s?weight capacity)|(\s\d*\,\d*\s?lb.?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?weight capacity)|(\s?\d*.?\d*\s?lb.?\s?d?i?s?t?r?i?b?u?t?e?d?\s?weight capacity)|(weight\s?capacity\s?of\s?\d+\s?lb)|(\s?\d+.?\d*\s?p?o?u?n?d?\s?weight capacity)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(m?a?x?i?m?u?m?\s?.{0,15}weight capacity.?\s?\d*.?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\s?\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(\s\d*\,\d*\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(m?a?x?i?m?u?m?\s?weight\s?capacity.?\s?\(?s?e?a?t?i?n?g?\)?\s?.?\s?\d*\s?lb)|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(Mini?m?u?m?.{0,15}\s?w?e?i?g?h?t?.?\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb)'   
df['matches_na'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

# regex_lb=r'(?i)(?<!weighs\s?)\d+.?\d*\s?lb'
# df['matches_lb'] = df['long_desc'].apply(lambda x: re_extract(regex_lb, x))

df_na=df[(df['matches_na'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]#&(df['matches_lb'].astype(str)!='[]')] 
df_na[curation_col]='n/a'
match_na=df_na[['external_id',curation_col]]
print(len(df_na))

168


In [12]:
rounding(df_newest,curation_col,'ac-km-z')

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, LONG_DESCRIPTION, matches, newest_matches, Q:weight, rounding]
Index: []

In [13]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_na))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnewest))

match_na
matchesnewest


In [14]:
print(len(df))
print(len(match_na)+len(matchesnewest))

1972
1828


In [15]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [19]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('Kimball MidWest', attribute, df, match_na)